**Formación en modelos predictivos (II)**

Con el fin de consolidar tus conocimientos en modelos predictivos, el ejercicio actual consiste en
crear un modelo para prever el mantenimiento a llevar a cabo en una serie de máquinas.
Se proporciona un dataset donde aparece las fechas en que ha fallado cada máquina. La idea, para
no complicar, es hacer mantenimiento en la máquina justo el día en que falla; asumiremos que, al
llevarlo a cabo a primera hora de la mañana, ésta ya no se estropeará.

Acceso al dataset: https://apioverstand.es/training/mantenimiento_modificado.csv

**1. Importamos librerias necesarias**

In [1]:
import pandas as pd
from datetime import datetime

**2. Leemos el dataset**

In [2]:
# Ruta al archivo CSV
archivo_csv = "mantenimiento_modificado.csv"

# Lee el archivo CSV y almacena los datos en un DataFrame de Pandas
data = pd.read_csv(archivo_csv)
data.head(4)

,ID Máquina,Tipo,Fallo,Fecha de Fallo 1,Fecha de Fallo 2,Fecha de Fallo 3,Fecha de Fallo 4,Fecha de Fallo 5
0,L47762,1,1,2022-10-19,2022-11-12,2022-12-06,2022-12-30,2023-01-23
1,L50530,1,1,2022-12-02,2022-12-26,2023-01-19,2023-02-12,2023-03-08
2,M23706,2,1,2023-01-16,2023-02-10,2023-03-07,2023-04-01,2023-04-26
3,L51911,1,1,2023-03-02,2023-03-26,2023-04-19,2023-05-13,2023-06-06


**3. Obtenemos informacion sobre una maquina determinada**
 (id, tipo, fallo, fechas, y diferencia entre las fechas)

In [3]:
def obtener_informacion_por_id(id_maquina):
    # Elimina espacios en blanco adicionales y convierte los nombres de las columnas a minúsculas
    data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_')

    # Busca la fila con la ID de la máquina
    fila = data[data['id_máquina'] == id_maquina]

    if not fila.empty:
        # Obtiene la información
        tipo = fila['tipo'].values[0]
        fallo = fila['fallo'].values[0]
        
        # Selecciona las columnas de fechas de fallo
        fechas_fallo = fila[['fecha_de_fallo_1', 'fecha_de_fallo_2', 'fecha_de_fallo_3', 'fecha_de_fallo_4', 'fecha_de_fallo_5']]
        
        # Formatea las fechas en el formato deseado
        fechas_fallo_formateadas = [datetime.strptime(fecha, "%Y-%m-%d").strftime("%d/%m/%Y") for fecha in fechas_fallo.values[0]]

        # Calcula y muestra los días transcurridos entre fechas
        resultados = []
        for i in range(len(fechas_fallo_formateadas) - 1):
            fecha1 = datetime.strptime(fechas_fallo.iloc[0, i], "%Y-%m-%d")
            fecha2 = datetime.strptime(fechas_fallo.iloc[0, i + 1], "%Y-%m-%d")
            diferencia_dias = (fecha2 - fecha1).days
            resultado = f"FECHA DE FALLO {i + 1}: {fechas_fallo_formateadas[i]} - Desde la fecha {i + 1} a la {i + 2} han pasado {diferencia_dias} días"
            resultados.append(resultado)

        # Imprime la información
        print(f"MAQUINA: {id_maquina}")
        print(f"TIPO: {tipo}")
        print(f"FALLO: {fallo}")
        for i, resultado in enumerate(resultados, start=1):
            print(resultado)
        print(f"FECHA DE FALLO {len(resultados) + 1}: {fechas_fallo_formateadas[-1]}")
    else:
        print("La ID introducida no existe en la base de datos")


id_maquina = "L47762"
obtener_informacion_por_id(id_maquina)

MAQUINA: L47762
TIPO: 1
FALLO: 1
FECHA DE FALLO 1: 19/10/2022 - Desde la fecha 1 a la 2 han pasado 24 días
FECHA DE FALLO 2: 12/11/2022 - Desde la fecha 2 a la 3 han pasado 24 días
FECHA DE FALLO 3: 06/12/2022 - Desde la fecha 3 a la 4 han pasado 24 días
FECHA DE FALLO 4: 30/12/2022 - Desde la fecha 4 a la 5 han pasado 24 días
FECHA DE FALLO 5: 23/01/2023


Podemos observar que la diferencia entre fechas es constante. En algunos casos 25 dias, en otros 24 y en otros 20, pero en cada caso siempre es constante.

Por tanto, haremos otra funcion que compruebe si esto ocurre en todos los registros, o por el contrario, hay registros cuya diferencia de fechas no sea constante.

In [4]:
def verificar_diferencia_constante_entre_fechas(data):
    # Elimina espacios en blanco adicionales y convierte los nombres de las columnas a minúsculas
    data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_')

    diferencias_constantes = True
    diferencias_registros = []

    for i, fila in data.iterrows():
        fechas_fallo = [fila['fecha_de_fallo_1'], fila['fecha_de_fallo_2'], fila['fecha_de_fallo_3'], fila['fecha_de_fallo_4'], fila['fecha_de_fallo_5']]
        diferencias = []

        for j in range(len(fechas_fallo) - 1):
            fecha1 = datetime.strptime(fechas_fallo[j], "%Y-%m-%d")
            fecha2 = datetime.strptime(fechas_fallo[j + 1], "%Y-%m-%d")
            diferencia = (fecha2 - fecha1).days
            diferencias.append(diferencia)

        diferencias_registros.append(diferencias)

    for i, diferencias in enumerate(diferencias_registros):
        if len(set(diferencias)) != 1:
            diferencias_constantes = False

    if diferencias_constantes:
        print("La diferencia de fechas en todos los registros es constante")
    else:
        print("La diferencia de fechas entre todos los registros NO es constante")

# Llama a la función para verificar las diferencias entre las fechas
verificar_diferencia_constante_entre_fechas(data)

La diferencia de fechas en todos los registros es constante


Podemos observar que en todos los registros la diferencia de fechas siempre es constante. Ahora veamos que relevancia tiene esto en el tipo y fallo de la maquina

Primero comprobaremos si hay alguna maquina que este duplicada

In [7]:
# Verifica si hay duplicados en la columna 'id_máquina'
duplicados = data['id_máquina'].duplicated(keep=False)  # El argumento 'keep=False' marca como True todas las ocurrencias duplicadas

# Comprueba si hay algún duplicado
if duplicados.any():
    print("Hay registros duplicados en la columna 'id_máquina':")
    registros_duplicados = data[duplicados]
    print(registros_duplicados)
else:
    print("No hay registros duplicados en la columna 'id_máquina'.")

Hay registros duplicados en la columna 'id_máquina':
   id_máquina  tipo  fallo fecha_de_fallo_1 fecha_de_fallo_2 fecha_de_fallo_3  \
30     M15854     2      3       2022-12-25       2023-01-10       2023-01-26   
31     M15854     2      1       2023-01-13       2023-02-07       2023-03-04   
32     L56264     1      2       2022-06-22       2022-07-17       2022-08-11   
33     L56264     1      1       2022-11-09       2022-12-03       2022-12-27   

   fecha_de_fallo_4 fecha_de_fallo_5  
30       2023-02-11       2023-02-27  
31       2023-03-29       2023-04-23  
32       2022-09-05       2022-09-30  
33       2023-01-20       2023-02-13  


Efectivamente, comprobamos que hay maquinas que estan duplicadas porque han tenido varios tipos de fallo. Por tanto, la frecuencia de averias parece ser que depende del tipo y del fallo.

Asi es que crearemos un codigo que nos diga la diferencia de fechas (o frecuencia de averias) en cada registro.

In [8]:
# Crear una copia de data para no modificar el DataFrame original
data_copy = data.copy()

In [9]:

# Función para calcular la diferencia de días para un registro
def calcular_diferencia_dias(fila):
    fechas_fallo = [fila['fecha_de_fallo_1'], fila['fecha_de_fallo_2'], fila['fecha_de_fallo_3'], fila['fecha_de_fallo_4'], fila['fecha_de_fallo_5']]
    diferencias = []

    for i in range(len(fechas_fallo) - 1):
        fecha1 = pd.to_datetime(fechas_fallo[i])
        fecha2 = pd.to_datetime(fechas_fallo[i + 1])
        diferencia = (fecha2 - fecha1).days
        diferencias.append(diferencia)

    return diferencias

Crearemos un nuevo dataframe que contenga la maquina, tipo, fallo y la frecuencia de las averias para poderlo analizar y encontrar la relacion existente entre dichas variables

In [10]:

# Calcular las diferencias de días para todos los registros
data_copy['dif_dias'] = data_copy.apply(calcular_diferencia_dias, axis=1)

# Crea una lista de diccionarios
data_list = []

for i, fila in data_copy.iterrows():
    id_maquina = fila['id_máquina']
    tipo_averia = fila['tipo']
    fallo = fila['fallo']
    dif_dias = calcular_diferencia_dias(fila)
    
    for dif in dif_dias:
        data_list.append({'id_máquina': id_maquina, 'tipo_av': tipo_averia, 'fallo': fallo, 'dif_dias': dif})

# Crear el DataFrame df_dif_dias a partir de la lista de diccionarios
df_dif_dias = pd.DataFrame(data_list)

# Reordenar las columnas
df_dif_dias = df_dif_dias[['id_máquina', 'tipo_av', 'fallo', 'dif_dias']]

# Eliminar los registros duplicados en todas sus columnas, si los hubiera. 
df_dif_dias = df_dif_dias.drop_duplicates().reset_index(drop=True)

# Exportar el nuevo DataFrame df_dif_dias a un archivo Excel
# df_dif_dias.to_excel('df_dif_dias.xlsx', index=False)

# Mostrar df_dif_dias
df_dif_dias.head(8)

,id_máquina,tipo_av,fallo,dif_dias
0,L47762,1,1,24
1,L50530,1,1,24
2,M23706,2,1,25
3,L51911,1,1,24
4,L56594,1,1,24
5,L51482,1,2,25
6,L51293,1,1,24
7,L48598,1,1,24


Creamos otro codigo que nos diga el patron existente entre las variables tipo-fallo y la variable diferencia de fechas (o frecuencia) usando el dataframe generado en la celda anterior (df_dif_dias)

In [11]:
# Agrupar el DataFrame por 'tipo_av' y 'fallo' y calcular la diferencia de días promedio
resultados = df_dif_dias.groupby(['tipo_av', 'fallo'])['dif_dias'].mean().reset_index()

# Crear un diccionario para almacenar el patrón encontrado
patron = {}

# Iterar a través de los resultados y verificar el patrón
for index, row in resultados.iterrows():
    tipo_av = row['tipo_av']
    fallo = row['fallo']
    dif_dias = row['dif_dias']
    
    if (tipo_av, fallo) not in patron:
        patron[(tipo_av, fallo)] = dif_dias
    else:
        if patron[(tipo_av, fallo)] != dif_dias:
            patron[(tipo_av, fallo)] = None

print('Patron detectado:')
# Función para formatear la descripción del patrón
def formato_descripcion(tipo_av, fallo, dif_dias):    
    if dif_dias is None:
        return f'El patrón no es constante para tipo {tipo_av} y fallo {fallo}'
    else:        
        return f'Para tipo {int(tipo_av)} y fallo {int(fallo)} La diferencia de días entre averias es {int(dif_dias)}'
    

# Mostrar el patrón encontrado
for key, value in patron.items():    
    print(formato_descripcion(key[0], key[1], value))

print("")
print('Variable patron que usaremos mas adelante: ')
patron

Patron detectado:
Para tipo 1 y fallo 1 La diferencia de días entre averias es 24
Para tipo 1 y fallo 2 La diferencia de días entre averias es 25
Para tipo 1 y fallo 3 La diferencia de días entre averias es 22
Para tipo 2 y fallo 1 La diferencia de días entre averias es 25
Para tipo 2 y fallo 3 La diferencia de días entre averias es 16
Para tipo 3 y fallo 1 La diferencia de días entre averias es 20

Variable patron que usaremos mas adelante: 


{(1.0, 1.0): 24.0,
 (1.0, 2.0): 25.0,
 (1.0, 3.0): 22.0,
 (2.0, 1.0): 25.0,
 (2.0, 3.0): 16.0,
 (3.0, 1.0): 20.0}

En el resultado anterior podemos observar que existe un patron muy definido. La frecuencia de las averias depende de la combinacion entre tipo y fallo.
Por tanto, para calcular la fecha de la siguiente averia, tomaremos como datos de entrada el tipo, el fallo y la ultima fecha. De esa forma, segun el tipo y fallo podremos saber la diferencia de dias (previamente almacenada en la variable patron) y se la añadiremos a la ultima fecha. 
Este dato nuevo se almacenara en fecha_de_fallo_6 y, junto con todos los demas datos se guardara en un df llamado *df_calendario*

In [12]:
# Copiar el DataFrame data a df_calendario
df_calendario = data.copy()

# Calcular la fecha_de_fallo_6 utilizando el diccionario patron
df_calendario['fecha_de_fallo_6'] = df_calendario.apply(
    lambda row: pd.to_datetime(row['fecha_de_fallo_5']) + pd.DateOffset(days=patron.get((row['tipo'], row['fallo']), 0)),
    axis=1
)

# Reordenar las columnas
df_calendario = df_calendario[['id_máquina', 'tipo', 'fallo', 'fecha_de_fallo_1', 'fecha_de_fallo_2', 'fecha_de_fallo_3', 'fecha_de_fallo_4', 'fecha_de_fallo_5', 'fecha_de_fallo_6']]

# Mostrar el DataFrame df_calendario
df_calendario.head(4)

,id_máquina,tipo,fallo,fecha_de_fallo_1,fecha_de_fallo_2,fecha_de_fallo_3,fecha_de_fallo_4,fecha_de_fallo_5,fecha_de_fallo_6
0,L47762,1,1,2022-10-19,2022-11-12,2022-12-06,2022-12-30,2023-01-23,2023-02-16
1,L50530,1,1,2022-12-02,2022-12-26,2023-01-19,2023-02-12,2023-03-08,2023-04-01
2,M23706,2,1,2023-01-16,2023-02-10,2023-03-07,2023-04-01,2023-04-26,2023-05-21
3,L51911,1,1,2023-03-02,2023-03-26,2023-04-19,2023-05-13,2023-06-06,2023-06-30


En el siguiente paso crearemos un codigo que nos confirme que el dataframe anterior tiene todas las fechas calculadas correctamente.
Lo haremos usando como referencia el dataframe *df_dif_dias*

In [13]:
# Antes que nada, añadimos la columna dif_dias porque la vamos a necesitar para esta tarea.
# Copia el DataFrame df_calendario a df_calendario_test
df_calendario_test = df_calendario.copy()

# Agrega la columna 'dif_dias' de df_dif_dias al DataFrame df_calendario_test
df_calendario_test['dif_dias'] = df_dif_dias['dif_dias']

# Muestra el DataFrame df_calendario_test
df_calendario_test.head(4)

,id_máquina,tipo,fallo,fecha_de_fallo_1,fecha_de_fallo_2,fecha_de_fallo_3,fecha_de_fallo_4,fecha_de_fallo_5,fecha_de_fallo_6,dif_dias
0,L47762,1,1,2022-10-19,2022-11-12,2022-12-06,2022-12-30,2023-01-23,2023-02-16,24
1,L50530,1,1,2022-12-02,2022-12-26,2023-01-19,2023-02-12,2023-03-08,2023-04-01,24
2,M23706,2,1,2023-01-16,2023-02-10,2023-03-07,2023-04-01,2023-04-26,2023-05-21,25
3,L51911,1,1,2023-03-02,2023-03-26,2023-04-19,2023-05-13,2023-06-06,2023-06-30,24


Por ultimo, crearemos un codigo que recorra todos los registros y compruebe que entre la *fecha_de_fallo5* y la *fecha_de_fallo_6* hay el numero de dias que dice la columna *dif_dias*, y si en algun registro esto no se cumple nos indique dicho registro.

In [14]:
# Este código recorre todos los registros en df_calendario_test, calcula la diferencia de días entre fecha_de_fallo_5 y fecha_de_fallo_6, 
# y compara esa diferencia con la columna dif_dias. Si todas las fechas coinciden, mostrará el mensaje "TODAS LAS FECHAS ESTÁN CORRECTAMENTE CALCULADAS". 
# Si alguna fecha no coincide, mostrará el mensaje "En el registro X la fecha_de_fallo_6 no coincide con la columna dif_dias", 
# donde X es el número de registro donde la condición no se cumple.

# Variable para verificar si todas las fechas coinciden
todas_las_fechas_correctas = True

# Recorre los registros de df_calendario_test
for index, row in df_calendario_test.iterrows():
    fecha_de_fallo_5 = pd.to_datetime(row['fecha_de_fallo_5'])
    fecha_de_fallo_6 = pd.to_datetime(row['fecha_de_fallo_6'])
    dif_dias = row['dif_dias']

    if (fecha_de_fallo_6 - fecha_de_fallo_5).days != dif_dias:
        todas_las_fechas_correctas = False
        print(f"En el registro {index} la fecha_de_fallo_6 no coincide con la columna dif_dias")

# Comprueba si todas las fechas coinciden y muestra el mensaje correspondiente
if todas_las_fechas_correctas:
    print("En todos los registros coinciden las fechas con la frecuencia de las averías, por tanto:")
    print("TODAS LAS FECHAS ESTÁN CORRECTAMENTE CALCULADAS")


En todos los registros coinciden las fechas con la frecuencia de las averías, por tanto:
TODAS LAS FECHAS ESTÁN CORRECTAMENTE CALCULADAS


In [15]:
df_calendario.head(3)

,id_máquina,tipo,fallo,fecha_de_fallo_1,fecha_de_fallo_2,fecha_de_fallo_3,fecha_de_fallo_4,fecha_de_fallo_5,fecha_de_fallo_6
0,L47762,1,1,2022-10-19,2022-11-12,2022-12-06,2022-12-30,2023-01-23,2023-02-16
1,L50530,1,1,2022-12-02,2022-12-26,2023-01-19,2023-02-12,2023-03-08,2023-04-01
2,M23706,2,1,2023-01-16,2023-02-10,2023-03-07,2023-04-01,2023-04-26,2023-05-21


Por ultimo, una vez ya estamos seguros que todas las fechas han sido calculadas correctamente, al ser un calendario, haremos una copia modificada de *df_calendario* pero ordenada en forma ascendente en su columna *fecha_de_fallo_6*, renombraremos esta columna por *fecha_prox_averia*, cambiaremos el formato de fecha *AAAA-MM-DD* por *DD-MM-AAAA*, resetearemos el index y, por ultimo, lo almacenaremos en una hoja excel que sera el resultado final para posteriores consultas.

In [16]:
# Crear una copia modificada de df_calendario y ordenarla por fecha_de_fallo_6
df_calendario_ordenado = df_calendario.copy()
df_calendario_ordenado = df_calendario_ordenado.sort_values(by='fecha_de_fallo_6', ascending=True)

# Renombrar la columna fecha_de_fallo_6 a fecha_prox_averia
df_calendario_ordenado.rename(columns={'fecha_de_fallo_6': 'fecha_prox_averia'}, inplace=True)

# Resetea el índice del DataFrame
df_calendario_ordenado.reset_index(drop=True, inplace=True)

# Cambiar el formato de las fechas a DD-MM-AAAA
df_calendario_ordenado['fecha_de_fallo_1'] = pd.to_datetime(df_calendario_ordenado['fecha_de_fallo_1']).dt.strftime('%d-%m-%Y')
df_calendario_ordenado['fecha_de_fallo_2'] = pd.to_datetime(df_calendario_ordenado['fecha_de_fallo_2']).dt.strftime('%d-%m-%Y')
df_calendario_ordenado['fecha_de_fallo_3'] = pd.to_datetime(df_calendario_ordenado['fecha_de_fallo_3']).dt.strftime('%d-%m-%Y')
df_calendario_ordenado['fecha_de_fallo_4'] = pd.to_datetime(df_calendario_ordenado['fecha_de_fallo_4']).dt.strftime('%d-%m-%Y')
df_calendario_ordenado['fecha_de_fallo_5'] = pd.to_datetime(df_calendario_ordenado['fecha_de_fallo_5']).dt.strftime('%d-%m-%Y')
df_calendario_ordenado['fecha_prox_averia'] = pd.to_datetime(df_calendario_ordenado['fecha_prox_averia']).dt.strftime('%d-%m-%Y')

# Exporta el DataFrame a un archivo Excel llamado calendario_averias.xlsx
# df_calendario_ordenado.to_excel('calendario_averias.xlsx', index=False)
print("CALENDARIO DEFINITIVO DE AVERIAS (ordenador por fecha)")
print("Puede verlo completo en el archivo calendario_averias.xlsx")
df_calendario_ordenado.head(8)

CALENDARIO DEFINITIVO DE AVERIAS (ordenador por fecha)
Puede verlo completo en el archivo calendario_averias.xlsx


,id_máquina,tipo,fallo,fecha_de_fallo_1,fecha_de_fallo_2,fecha_de_fallo_3,fecha_de_fallo_4,fecha_de_fallo_5,fecha_prox_averia
0,L52242,1,1,20-04-2022,14-05-2022,07-06-2022,01-07-2022,25-07-2022,18-08-2022
1,L47230,1,2,23-05-2022,17-06-2022,12-07-2022,06-08-2022,31-08-2022,25-09-2022
2,L50864,1,1,28-05-2022,21-06-2022,15-07-2022,08-08-2022,01-09-2022,25-09-2022
3,L47428,1,1,09-06-2022,03-07-2022,27-07-2022,20-08-2022,13-09-2022,07-10-2022
4,L51228,1,2,13-06-2022,08-07-2022,02-08-2022,27-08-2022,21-09-2022,16-10-2022
5,L54258,1,1,19-06-2022,13-07-2022,06-08-2022,30-08-2022,23-09-2022,17-10-2022
6,L56264,1,2,22-06-2022,17-07-2022,11-08-2022,05-09-2022,30-09-2022,25-10-2022
7,L55787,1,1,05-07-2022,29-07-2022,22-08-2022,15-09-2022,09-10-2022,02-11-2022


Ahora, aparte de la hoja excel, podemos consultar el calendario de averias de una maquina determinada 

In [18]:
# Introduzca la ID de la maquina a consultar
id_maquina = 'M15854'
# id_maquina = 'L47762'
registros = df_calendario_ordenado[df_calendario_ordenado['id_máquina'] == id_maquina]
registros

,id_máquina,tipo,fallo,fecha_de_fallo_1,fecha_de_fallo_2,fecha_de_fallo_3,fecha_de_fallo_4,fecha_de_fallo_5,fecha_prox_averia
21,M15854,2,3,25-12-2022,10-01-2023,26-01-2023,11-02-2023,27-02-2023,15-03-2023
30,M15854,2,1,13-01-2023,07-02-2023,04-03-2023,29-03-2023,23-04-2023,18-05-2023
